In [1]:
import pandas as pd
import numpy as np

ad = pd.read_csv("Advertising.csv",usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [2]:
x = df.drop("sales",axis=1)
x[:10]

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4
5,8.7,48.9,75.0
6,57.5,32.8,23.5
7,120.2,19.6,11.6
8,8.6,2.1,1.0
9,199.8,2.6,21.2


In [8]:
y = df["sales"]

In [9]:
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict

In [10]:
#train-test veri seti bölme işlemi

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20,random_state=42)


In [11]:
x_train.shape

(160, 3)

In [12]:
x_test.shape

(40, 3)

In [13]:
y_train.shape

(160,)

In [14]:
y_test.shape

(40,)

In [16]:
#statsmodels ile model kurma
import statsmodels.api as sm

lm = sm.OLS(y_train,x_train)

In [17]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     2935.
Date:                Tue, 01 Oct 2019   Prob (F-statistic):          1.28e-137
Time:                        12:58:57   Log-Likelihood:                -336.65
No. Observations:                 160   AIC:                             679.3
Df Residuals:                     157   BIC:                             688.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#scikit-learn ile model kurma


from sklearn.linear_model import LinearRegression

In [19]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)

In [20]:
model.intercept_

2.979067338122629

In [21]:
model.coef_

array([0.04472952, 0.18919505, 0.00276111])

In [22]:
#Tahmin

#Model Denklemi:

#Sales = 2.97 + TV*0.04 + radio*0.18 + newspaper*0.002

#Örneğin 30 birim TV harcaması,10 birim radio harcaması,40 birimde gazete harcaması olduğunda
#satışların tahmini değeri ne olur ?

In [23]:
yeni_veri = [[30],[10],[40]]
yeni_veri=pd.DataFrame(yeni_veri).T

In [24]:
model.predict(yeni_veri)

array([6.32334798])

In [26]:
from sklearn.metrics import mean_squared_error,r2_score

#model.predict(x_train) : y şapka değerleri

rmse = np.sqrt(mean_squared_error(y_train,model.predict(x_train)))

In [27]:
rmse

1.644727765644337

In [29]:
rmse = np.sqrt(mean_squared_error(y_test,model.predict(x_test)))

In [30]:
rmse

1.7815996615334502

In [31]:
#Model Tuning/Model Doğrulama

x = df.drop('sales',axis = 1)
y = df['sales']

#random_state parametresi her değiştirildiğinde rmse ve r2 değerleri değişmektedir.
#Bunun sebebi verinin her seferinde farklı bir kısmının alınmasıdır.Veri setitinin
#hangi parçasını almamız gerektiğini bulmamız gerekiyor.Bu sorunu ortadan kaldırmak
#için cross validation yöntemi kullanılır.
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

lm = LinearRegression()
model = lm.fit(x_train,y_train)

In [32]:
np.sqrt(mean_squared_error(y_train,model.predict(x_train)))

1.644727765644337

In [33]:
model.score(x_train,y_train)

0.8957008271017818

In [37]:
cross_val_score(model,x,y,cv=10,scoring = "r2")

array([0.87302696, 0.8581613 , 0.92968723, 0.89013272, 0.93146498,
       0.93138735, 0.7597901 , 0.91217097, 0.83891753, 0.92882311])

In [38]:
cross_val_score(model,x,y,cv=10,scoring = "r2").mean()

0.8853562237979616

In [39]:
#doğrulanmış r2
cross_val_score(model,x_train,y_train,cv=10,scoring = "r2").mean()

0.791354859691634

In [44]:
cross_val_score(model,x_test,y_test,cv=10,scoring = "r2").mean()

0.7007204329162284

In [41]:
#doğrulanmış mse
cross_val_score(model,x_train,y_train,cv=10,scoring = "neg_mean_squared_error")

array([-5.57303426, -2.86235681, -2.06504684, -1.09186983, -1.66159795,
       -2.50694042, -2.92821679, -2.01207197, -7.2250041 , -1.66156243])

In [45]:
cross_val_score(model,x_test,y_test,cv=10,scoring = "neg_mean_squared_error")

array([-2.56888903, -2.59836095, -4.17513172, -4.73447809, -2.07237581,
       -1.81649898, -3.62864463, -4.35649927, -5.03508511, -4.02198093])

In [42]:
#doğrulanmış rmse
np.sqrt(-cross_val_score(model,x_train,y_train,cv=10,scoring = "neg_mean_squared_error"))

array([2.36072749, 1.69185011, 1.43702708, 1.04492575, 1.28902985,
       1.58333206, 1.71120332, 1.41847523, 2.68793677, 1.28901607])

In [46]:
np.sqrt(-cross_val_score(model,x_test,y_test,cv=10,scoring = "neg_mean_squared_error"))

array([1.60277541, 1.61194322, 2.04331391, 2.17588559, 1.43957487,
       1.34777557, 1.90490016, 2.08722286, 2.24389953, 2.0054877 ])

In [43]:
#gerçek/doğrulanmış eğitim hatası(rmse)

np.sqrt(-cross_val_score(model,x_train,y_train,cv=10,scoring = "neg_mean_squared_error")).mean()

1.6513523730313335

In [47]:
np.sqrt(-cross_val_score(model,x_test,y_test,cv=10,scoring = "neg_mean_squared_error")).mean()

1.8462778823997095